<a href="https://colab.research.google.com/github/theventurecity/data-toolkit/blob/master/Full_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://theventure.city"><img src="https://github.com/theventurecity/data-toolkit/blob/master/img/tvc_horiz_junglegreen.png?raw=true" alt='TheVentureCity' style="width: 400px;"></a>

# Full Pipeline
1. Extract raw event log data from a CSV
1. Transform the raw data into analysis dataframes
    - Growth Accounting
    - Cohort Analysis
    - Engagement
1. Load the transformed data into Google Sheets
1. Visualize insights in Google Data Studio

## Before you begin

- This notebook is shared in Google Colaboratory with read-only access. To run this notebook yourself, first click "**Open in Playground**" in the toolbar above. That will create a separate instance that you can run and/or save a copy of to your own Google Drive.

- To run each cell, hit **Shift-Enter**, which will run the contents of the active cell and move to the next cell. This includes the markup cells (such as this one).

- When you run the first block of Python code, you will get a message that says, "**Warning: This notebook was not authored by Google.**" Please be aware that we are **NOT** accessing your data shared with Google or reading data and credentials from other sessions. This notebook reads data from GitHub and writes to a Google Sheet that only you have access to and can control. We recommend you click the box to "**Reset all runtimes before running**" for extra information security.

## Import relevant Python libraries

In [77]:
### To run this with functions from TheVentureCity's GitHub repository,
### clone the repository to the Google Colaboratory runtime environment
from importlib.machinery import SourceFileLoader
!git clone https://github.com/theventurecity/data-toolkit.git /tmp/theventurecity

### Installs all the necessary library versions for this environment
!pip install -r /tmp/theventurecity/requirements.txt

### Pandas to read the data from its source and manipulate it in memory
import pandas as pd

### The IPython.display library allows us to embed an iFrame within this
### notebook
from IPython.display import IFrame

### Importing tvc_transform and tvc_load_colab, which contain pre-existing
### functions rather than having to define them inline with the notebook
### THIS IS ONLY APPLICABLE IF YOU WANT TO ACCESS THOSE FUNCTIONS ###
!mv /tmp/theventurecity/python/tvc_transform.py tvc_transform.py
!mv /tmp/theventurecity/python/tvc_load_colab.py tvc_load_colab.py
!rm -r /tmp/theventurecity
tvct = SourceFileLoader('tvc_transform', 'tvc_transform.py').load_module()
tvcl = SourceFileLoader('tvc_load_colab', 'tvc_load_colab.py').load_module()

Cloning into '/tmp/theventurecity'...
remote: Enumerating objects: 380, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 380 (delta 0), reused 0 (delta 0), pack-reused 377
Receiving objects: 100% (380/380), 63.64 MiB | 20.17 MiB/s, done.
Resolving deltas: 100% (234/234), done.
Updating files: 100% (28/28), done.


## 1. Extract raw event log data from a CSV

This example uses a data file for a sample company from our GitHub repository called ServBiz. In this step we read the data file into memory as a Pandas dataframe we name "t."

In [78]:
filename = 'https://raw.githubusercontent.com/theventurecity/analytics/master/data/ServBiz_transactions.csv'
t = pd.read_csv(filename)
t.tail(10)

,client_id,date,value_usd,segment
420781,27902A,2024-04-30,8.75,Enterprise
420782,34181A,2024-04-30,18.97,SMB
420783,30168A,2024-04-30,17.73,SMB
420784,30844A,2024-04-30,19.98,SMB
420785,35815A,2024-04-30,17.98,SMB
420786,16958A,2024-04-30,17.45,SMB
420787,13090A,2024-04-30,13.48,SMB
420788,19162A,2024-04-30,13.64,Enterprise
420789,28409A,2024-04-30,14.72,SMB
420790,12080A,2024-04-30,18.32,SMB


## 2. Transform the raw data into analysis dataframes
### 2.1 Create DAU Decorated and MAU Decorated "building block" dataframes
**Note**: For a more detailed discussion about creating the DAU and DAU Decorated dataframes, complete with inline code, visit [Create the DAU Decorated Data Set](https://colab.research.google.com/drive/12uehG2EcIqxcTazKs-pNQRTQSckllOmE)
#### 2.1.1 Create Daily Active Users (DAU) dataframe
The **DAU** dataframe aggregates all activity by user and day.

In [79]:
# Run the create_dau_df function and show the first ten rows of the resulting dataframe
dau = tvct.create_dau_df(t,
                         user_id = 'client_id',
                         activity_date = 'date',
                         inc_amt = 'value_usd',
                         segment_col = 'segment'
                        )
dau.head(10)

,user_id,activity_date,segment,inc_amt
0,10000A,2020-12-14,SMB,11.75
1,10001A,2020-12-14,SMB,13.75
2,10001A,2021-01-02,SMB,7.50
3,10001A,2021-01-22,SMB,18.00
4,10001A,2022-06-05,SMB,6.25
5,10001A,2023-02-08,SMB,8.75
6,10002A,2020-12-14,Enterprise,11.75
7,10002A,2020-12-26,Enterprise,12.25
8,10002A,2021-01-16,Enterprise,12.25
9,10002A,2021-01-23,Enterprise,12.25


#### 2.1.2 Calculate First Date and DAU Decorated dataframes
The create_dau_decorated_df function calls the create_first_dt_df if no first_dt dataframe is specified

In [80]:
# Run the create_dau_decorated_df function and show the first ten rows of the resulting dataframe
dau_decorated = tvct.create_dau_decorated_df(dau)
dau_decorated.head(10)

Creating DAU Decorated dataframe
Creating first_dt dataframe


,user_id,activity_date,segment,inc_amt,first_dt,first_week,first_month
0,10000A,2020-12-14,SMB,11.75,2020-12-14,2020-12-14/2020-12-20,2020-12
1,10001A,2020-12-14,SMB,13.75,2020-12-14,2020-12-14/2020-12-20,2020-12
2,10001A,2021-01-02,SMB,7.50,2020-12-14,2020-12-14/2020-12-20,2020-12
3,10001A,2021-01-22,SMB,18.00,2020-12-14,2020-12-14/2020-12-20,2020-12
4,10001A,2022-06-05,SMB,6.25,2020-12-14,2020-12-14/2020-12-20,2020-12
5,10001A,2023-02-08,SMB,8.75,2020-12-14,2020-12-14/2020-12-20,2020-12
6,10002A,2020-12-14,Enterprise,11.75,2020-12-14,2020-12-14/2020-12-20,2020-12
7,10002A,2020-12-26,Enterprise,12.25,2020-12-14,2020-12-14/2020-12-20,2020-12
8,10002A,2021-01-16,Enterprise,12.25,2020-12-14,2020-12-14/2020-12-20,2020-12
9,10002A,2021-01-23,Enterprise,12.25,2020-12-14,2020-12-14/2020-12-20,2020-12


Combining the basic DAU data with the first date, week, and month for each user, **the DAU Decorated dataframe is our basic building block for many different analyses**. It allows us to use user-level data to inspect engagement, retention, and growth accounting.
#### 2.1.3 Calculate MAU Decorated dataframe

In [81]:
# Run create_xau_decorated_df for MAU's (using 'month'), unsegmented
mau_decorated = tvct.create_xau_decorated_df(dau_decorated, 'month', use_segment=False)
mau_decorated.tail(10)

Creating Monthly Active Users Decorated dataframe


,Month_Year,user_id,inc_amt,first_month,Next_Month_Year
93545,2024-04,9794A,24.72,2020-11,2024-05
93546,2024-04,9808A,85.12,2020-11,2024-05
93547,2024-04,9833A,17.98,2020-11,2024-05
93548,2024-04,9868A,65.75,2020-12,2024-05
93549,2024-04,9876A,19.48,2020-12,2024-05
93550,2024-04,9902A,39.76,2020-12,2024-05
93551,2024-04,9952A,64.20,2020-12,2024-05
93552,2024-04,9986A,131.22,2020-12,2024-05
93553,2024-04,9989A,21.28,2020-12,2024-05
93554,2024-04,9995A,106.40,2020-12,2024-05


### 2.2 Transform into a Growth Accounting analysis dataframe
**Note**: The function called in this section is different than what appears in the [Mini-Pipeline: Growth Accounting](https://colab.research.google.com/drive/1moHa4Mcycwsz7Fq6T_5Zou1Zunt0afiI), which was condensed for brevity. The resultant dataset is therefore adjusted to write only the relevant columns to Google Sheets.

In [82]:
TIME_PER = 'month'
USE_SEGMENT = False
KEEP_LAST_PER = True
DATE_LIMIT = None
INCL_ZERO_INC = False

user_ga, rev_ga = tvct.create_growth_accounting_dfs(mau_decorated,
                                                    time_period=TIME_PER,
                                                    use_segment=USE_SEGMENT,
                                                    keep_last_period=KEEP_LAST_PER,
                                                    date_limit=DATE_LIMIT,
                                                    include_zero_inc=INCL_ZERO_INC)
user_ga_with_ratios = tvct.calc_user_ga_ratios(user_ga,
                                               time_period=TIME_PER,
                                               use_segment=USE_SEGMENT,
                                               growth_rate_periods=12)
user_ga_with_ratios.tail(10)

Creating Growth Accounting dataframes


,Month_Year,Monthly Active Users,Retained Users,New Users,Resurrected Users,Churned Users,segment,Users BOP,MoM User Retention,User Quick Ratio,User CMGR12,Growth Threshold,User CMGR12 Target
36,2023-07,3248,2505,435,308,-555,All,3060.0,0.818627,1.338739,0.031717,1.0,0.1
37,2023-08,3215,2631,356,228,-617,All,3248.0,0.810037,0.946515,0.025557,1.0,0.1
38,2023-09,3355,2694,372,289,-521,All,3215.0,0.837947,1.268714,0.033168,1.0,0.1
39,2023-10,3521,2813,438,270,-542,All,3355.0,0.838450,1.306273,0.035674,1.0,0.1
40,2023-11,3609,2929,412,268,-592,All,3521.0,0.831866,1.148649,0.036914,1.0,0.1
41,2023-12,3946,3048,569,329,-561,All,3609.0,0.844555,1.600713,0.038767,1.0,0.1
42,2024-01,4130,3308,530,292,-638,All,3946.0,0.838317,1.288401,0.036728,1.0,0.1
43,2024-02,4447,3486,687,274,-644,All,4130.0,0.844068,1.492236,0.043886,1.0,0.1
44,2024-03,4686,3778,603,305,-669,All,4447.0,0.849562,1.357250,0.040949,1.0,0.1
45,2024-04,4736,3889,570,277,-797,All,4686.0,0.829919,1.062735,0.038235,1.0,0.1


### 2.3 Transform into a Cohort Analysis dataframe
Create the MAU Cohorts dataframe, using MAU Decorated as an input

In [83]:
mau_cohorts = tvct.create_xau_cohort_df(mau_decorated, 'month')
mau_cohorts.tail(10)

,first_month,Month_Year,Months Since First,inc_amt,cust_ct,cohort_cust_ct,cum_inc_amt,cum_inc_per_cohort_cust,cust_ret_pct
1066,2023-12-01,2023-12-01,0,23029.76,569,569,23029.76,40.474095,1.000000
1067,2023-12-01,2024-01-01,1,25563.56,381,569,48593.32,85.401265,0.669596
1068,2023-12-01,2024-02-01,2,22518.13,321,569,71111.45,124.976186,0.564148
1069,2023-12-01,2024-03-01,3,23657.35,298,569,94768.80,166.553251,0.523726
1071,2024-01-01,2024-01-01,0,19143.70,530,530,19143.70,36.120189,1.000000
1072,2024-01-01,2024-02-01,1,19601.45,362,530,38745.15,73.104057,0.683019
1073,2024-01-01,2024-03-01,2,18878.39,312,530,57623.54,108.723660,0.588679
1075,2024-02-01,2024-02-01,0,22804.65,687,687,22804.65,33.194541,1.000000
1076,2024-02-01,2024-03-01,1,28401.44,469,687,51206.09,74.535793,0.682678
1078,2024-03-01,2024-03-01,0,21823.96,603,603,21823.96,36.192305,1.000000


### 2.4. Transform into Engagement dataframes
#### 2.4.1 Calculate User Periodic Usage
Now that we have the "DAU Decorated" data frame, we can use it to calculate engagement metrics since they are super-important for an early-stage startup. The **calc_user_periodic_usage** function below calculates the number of active days for each user in a 28-day window of time. It outputs a dataframes that holds the active day count for every user, sorted by their inc_amt.

In [84]:
### Run calc_user_periodic_usage and display the first 10 rows
#user_daily_usage = tvct.calc_user_periodic_usage(dau_decorated,
#                                            'day',
#                                            dau_decorated['activity_date'].max(),
#                                            window_days=28,
#                                            breakouts=[2, 4],
#                                            use_segment=False
#                                           )
#user_daily_usage.head(10)

The calc_user_periodic_usage function above serves as a key helper function for the functions below that summarize its data into the data that will be used for the visualizations.

#### 2.4.2 Calculate the DAU Histogram
The DAU histogram puts each user into a bin according to the number of active days they have in the 28-day window. It outputs the user count in each bin and also counts the average number of days active across all users during the window.

In [85]:
dau_hist_L28 = tvct.calc_xau_hist(dau_decorated,
                                  'day',
                                  dau_decorated['activity_date'].max(),
                                  window_days=28,
                                  use_segment=False)
dau_hist_L28

,active_days_bin,user_count,avg_days_active
0,1,921,4.072579
1,2,1006,4.072579
2,3,487,4.072579
3,4,1148,4.072579
4,5,104,4.072579
5,6,165,4.072579
6,7,135,4.072579
7,8,305,4.072579
8,9,52,4.072579
9,10,116,4.072579


#### 2.4.3 Calculate Rolling DAU/MAU Ratios over Time
The DAU Histogram shown above helps us understand engagement during a fixed 28-day period. What if we could calculate its key attributes to look at engagement trends over time? We can do so by performing a similar analysis and looping through it on a daily basis. In doing so, we can build out a dataframe that contains engagement metrics for every 28-day period over the life of the company.

In [86]:
# Run rolling_dau_mau and show the last ten rows (the most recent ones)
rolling_dau_mau = tvct.create_xau_window_df(dau_decorated, breakouts = [2, 4, 8, 12])
rolling_dau_mau.tail(10)

1372 total days to process...
Processing day 0 of 1372...
Processing day 100 of 1372...
Processing day 200 of 1372...
Processing day 300 of 1372...
Processing day 400 of 1372...
Processing day 500 of 1372...
Processing day 600 of 1372...
Processing day 700 of 1372...
Processing day 800 of 1372...
Processing day 900 of 1372...
Processing day 1000 of 1372...
Processing day 1100 of 1372...
Processing day 1200 of 1372...
Processing day 1300 of 1372...
Finished processing all 1372 days!


,index,active_days,1d+ users,dau_window_ratio,window_frequency,2d+ users,2d+ users / total 28d users,4d+ users,4d+ users / total 28d users,8d+ users,8d+ users / total 28d users,12d+ users,12d+ users / total 28d users,window_end_dt
0,0,18775,4690,0.142971,4.003198,3719,0.792964,2185,0.465885,682,0.145416,186,0.039659,2024-04-21
0,0,18789,4694,0.142956,4.002769,3711,0.790584,2187,0.465914,681,0.145079,186,0.039625,2024-04-22
0,0,18821,4695,0.143169,4.008733,3715,0.791267,2200,0.468584,677,0.144196,186,0.039617,2024-04-23
0,0,18844,4709,0.142918,4.001699,3720,0.789977,2214,0.470164,678,0.143980,186,0.039499,2024-04-24
0,0,18853,4712,0.142895,4.001061,3722,0.789898,2216,0.470289,678,0.143888,186,0.039474,2024-04-25
0,0,18871,4697,0.143488,4.017671,3722,0.792421,2221,0.472855,680,0.144773,188,0.040026,2024-04-26
0,0,18865,4691,0.143626,4.021531,3724,0.793861,2216,0.472394,674,0.143679,187,0.039864,2024-04-27
0,0,18868,4684,0.143864,4.028181,3719,0.793980,2219,0.473740,680,0.145175,186,0.039710,2024-04-28
0,0,18901,4675,0.144393,4.042995,3728,0.797433,2229,0.476791,681,0.145668,188,0.040214,2024-04-29
0,0,18966,4657,0.145449,4.072579,3736,0.802233,2243,0.481641,691,0.148379,188,0.040369,2024-04-30


## 3. Load the transformed data into Google Sheets
**Note**: For a more detailed discussion about loading data into Google Sheets, complete with inline code, visit [Section 3 of Mini-Pipeline: Cohort Analysis](https://colab.research.google.com/drive/1oYy-wJl6VZFgOsv8uw7iGChQxUjrR5rf#scrollTo=hzG-fUwkyM9K)
### 3.1 Establish connection to Google Sheets for writing output files
The first time you run the cell below, or after some time of inactivity, you will be asked to click on a link. That link will take you to a new tab that will authorize this script to write to Google Sheets spreadsheets in your Google Account. To enable this feature, copy the code you get into the box below and hit Enter.

In [87]:
gc = tvcl.google_authenticate()

In [88]:
### Be sure to set this value to refer to your Google Sheets workbook
GOOGLE_SHEET_KEY = '1-XnO_eWkRwX-E1fiA2Jkbe3kJvoyoPFsdeW7vnF6zS0'

In [89]:
print('https://docs.google.com/spreadsheets/d/' + GOOGLE_SHEET_KEY)

https://docs.google.com/spreadsheets/d/1-XnO_eWkRwX-E1fiA2Jkbe3kJvoyoPFsdeW7vnF6zS0


### 3.2 Write the output files to Google Sheets

In [90]:
### Execute this function to write the data in the dataframe to the google sheet
### and tab name specified using the gc Google credentials
tvcl.write_to_google_sheet(user_ga_with_ratios,
                           GOOGLE_SHEET_KEY,
                           'MAU Growth Accounting',
                           gc)

In [91]:
### Execute this function to write the data in the dataframe to the google sheet
### and tab name specified using the gc Google credentials
tvcl.write_to_google_sheet(mau_cohorts,
                           GOOGLE_SHEET_KEY,
                           'MAU Retention by Cohort',
                           gc)

In [92]:
### Execute this function to write the data in the dataframe to the google sheet
### and tab name specified using the gc Google credentials
tvcl.write_to_google_sheet(dau_hist_L28,
                           GOOGLE_SHEET_KEY,
                           'DAU Histogram L28',
                           gc)

In [93]:
### Execute this function to write the data in the dataframe to the google sheet
### and tab name specified using the gc Google credentials
tvcl.write_to_google_sheet(rolling_dau_mau,
                           GOOGLE_SHEET_KEY,
                           'Rolling DAU/MAU',
                           gc)

## 4. Visualize insights in Google Data Studio
A Google DataStudio dashboard preconfigured to read from the Google Sheet created above to visualize the data is [available at this link](https://datastudio.google.com/open/1xjS__Q6ZUXuUUARkgRvY4spYUw1ePksV) or by clicking on the Google DataStudio logo at the bottom of the chart embedded below. It is available in read-only mode for you to copy, link to your own Google Sheet tabs, and see your own data visualized.
### 4.1 Show MAU
The first chart shows MAU with the three main active categories broken out:

- Retained Users are in blue
- New Users are in light green
- Resurrected Users are in darker green

Together these three categories add up to the total MAU for each month. The line snaking through the chart is the compound monthly growth rate calculated over a 12-month period (CMGR12). We see that it starts off high when the growth starts from a low base. But over time, the CMGR12 stabilizes around 4%. The User GMGR12 Target line of 10% serves as an example of a healthy growth rate for a successful startup, though this can vary depending on the situation.

In [94]:
IFrame('https://datastudio.google.com/embed/reporting/1xjS__Q6ZUXuUUARkgRvY4spYUw1ePksV/page/YVKk',
       width=600,
       height=450)

### 4.2 Show MAU Growth Accounting
The chart below is similar to the last one in that it still shows the same New and Resurrected user counts in light green and darker green, respectively. This chart removes the retained user count in favor of churned users in orange (expressed as a negative number). It also removes the CMGR12 line in favor of two new lines: the User Quick Ratio (in black) and MoM User Retention rate (in blue).

- User Quick Ratio = -1 * (New Users + Resurrected Users) / Churned Users
  - In other words, it is the ratio of the green bars at the top and the orange bar on the bottom
  - The horizontal line where the Quick Ratio = 1 serves as the demarcation line between growth and shrinkage
  - A Quick Ratio > 1 means that MAUs grew from last month to this month
  - A Quick Ratio < 1 means that MAUs shrank from last month to this month
  - For more about the Quick Ratio, visit our blog post [Quick Ratio as a Shortcut to Understand Product Growth](https://medium.com/theventurecity/quick-ratio-as-a-shortcut-to-understand-product-growth-ae60212bd371)
- MoM Retention Rate = Retained Users / Users BOP
  - Users BOP ("beginning of period") are the same as last month's users as calculated in the calc_user_ga_ratios function

In [95]:
IFrame('https://datastudio.google.com/embed/reporting/1xjS__Q6ZUXuUUARkgRvY4spYUw1ePksV/page/7eKk',
       width=600,
       height=450)

### 4.3 Monthly Cohort User Retention
One way to think about cohort user retention is based on the fact that, typically, not all users are retained from Month 0 to Month 1, and then to Month 2, 3, 4, etc. The chart below depicts this in the downward curve approaching. If you mouseover the bars, we can see that the January 2018 cohort at the farthest right is at 23.7% user retention in Month 13. Compare that to the 60.94% figure for that same cohort in Month 1.

Sometimes this data is depicted as downward curving lines, one for each cohort. We prefer this visual because (a) the lines often end up looking jumbled and spaghetti-like; and (b) it allows us to see the trend in each Months Since First's retention. For example, the most recent Month 1 retention of 66.55% for the January 2019 cohort is down from the Month 1 peak of 72.8% percent for the July 2018 cohort. In this way, we can see the general downward slope that the spaghetti lines would give us, but also see the trends within each Months Since First, for a superior visual.

In [96]:
IFrame('https://datastudio.google.com/embed/reporting/1xjS__Q6ZUXuUUARkgRvY4spYUw1ePksV/page/EOyj',
       width=600,
       height=450)

### 4.4 Cumulative Customer Revenue LTV by Monthly Cohort
The visual below shows the cumulative inc_amt *per original cohort customer* that each cohort has generated in the time it has been active. These lines give a look at how well a group of ServBiz's users keeps generating revenue after they are initially acquired. This gives us an empirical way to look at customer long-term value (LTV) over a range of possible scenarios. For example, if you mouseover 16 Months Since First in the chart below, the four cohorts that have been around that long range from `$`440.47 to `$`588.87 per cohort customer. We can use that as a 16-month LTV range for projection or for calculating LTV-to-CAC ratio.

Another feature of the curves below is that, while they are slighly bending downward, the rate of change in the slope is very gradual. This implies that ServBiz's customers are active for a long period of time, meaning that the time horizon for calculating LTV may be extended and that these customers are quite value in the long-term.

In [97]:
IFrame('https://datastudio.google.com/embed/reporting/1xjS__Q6ZUXuUUARkgRvY4spYUw1ePksV/page/pVBk',
       width=600,
       height=450)

### 4.5 User DAU Histogram
The chart below shows the number of users in each active days bin to illustrate the distribution of usage. The more users come back to the service on multiple days, the more engaged they are. We see that the highest bar is at 4 active days, or once per week. We also see another, smaller spike at 8 active days, or twice per week. The 4.1 figure for "Average Days Active in Last 28" is another way of expressing the DAU/MAU ratio. The DAU/MAU ratio in this case is 4.1/28, or 0.146, or 14.6%. That means users are active on 14.6% percent of possible days. We think it is easier to understand what this number means by thinking of it as average active days out of 28 rather than as a ratio/percentage.

In [98]:
IFrame('https://datastudio.google.com/embed/reporting/1xjS__Q6ZUXuUUARkgRvY4spYUw1ePksV/page/IEWi',
       width=600,
       height=450)

### 4.6 Rolling Active Days per 28
The chart below takes the 4.1 Average Active Days in Last 28 from the last chart--which was calculated from the most recent date in the event log, and plots the trend of that number over time. By looking at this number's trend, we can judge whether engagement is getting better, getting worse, or staying the same. In the case of ServBiz, engagement has leveled out around 4.0 over the last year-and-a-half or so after showing steady improvement in the first few years of the business. The exception is the regular seasonal dips when usage frequency wanes for a few weeks.

In [99]:
IFrame('https://datastudio.google.com/embed/reporting/1xjS__Q6ZUXuUUARkgRvY4spYUw1ePksV/page/hXXk',
       width=600,
       height=450)

### 4.7 Rolling 28-Day Multi-Day Users
A richer display of the distibution of active usage in effect combines the histogram from 4.1 above with the rolling trends from 4.2. In effect, we are turning the histogram on its side and dragging it through time to see the percentage of total active users above various thresholds. In the example below we have plotted four lines. They show the percentage of active users with...

- 2+ active days out of the previous 28, or once at least every other week (about 80% in the most recent 28-day window; see table below the chart)
- 4+ active days, or at least once a week (about 48%)
- 8+ active days, or at least twice a week (about 15%)
- 12+ active days, or at least 3x per week (about 4%)

Going with more than 12 days would make for a lousy visual with a horizontal line bouncing off zero.

In this visual we see that the percentage of users with 2+ days out of 28 continues to increase a bit over time. It is the more frequent categories (4+, 8+) that are flat and dragging down the overall average seen in 4.2.

In [100]:
IFrame('https://datastudio.google.com/embed/reporting/1xjS__Q6ZUXuUUARkgRvY4spYUw1ePksV/page/qdXk',
       width=600,
       height=450)

In [101]:
rolling_dau_mau.tail(1)

,index,active_days,1d+ users,dau_window_ratio,window_frequency,2d+ users,2d+ users / total 28d users,4d+ users,4d+ users / total 28d users,8d+ users,8d+ users / total 28d users,12d+ users,12d+ users / total 28d users,window_end_dt
0,0,18966,4657,0.145449,4.072579,3736,0.802233,2243,0.481641,691,0.148379,188,0.040369,2024-04-30
